In [ ]:
from helpers_3d import *
#import seaborn as sns
from tqdm import trange

df_train = pd.read_csv('./train-valid-splits-video/train.csv')
df_valid = pd.read_csv('./train-valid-splits-video/valid.csv')

train_loader = load_data(df_train, 18, 16)
valid_loader = load_data(df_valid, 18, 16)

options = {
    "model_depth": 50,
    "model": 'resnet',
    "n_classes": 400,
    "n_finetune_classes": 5,
    "resnet_shortcut": 'B',
    "sample_size": (576,704),
    "sample_duration": 16,
    "pretrain_path": '../3D-ResNets-PyTorch/resnet-50-kinetics.pth',
    "no_cuda": False,
    "arch": 'resnet-50',
    "ft_begin_index": 0
}

opts = namedtuple("opts", sorted(options.keys()))
myopts2 = opts(**options)
model, parameters = generate_model(myopts2)

In [ ]:
adaptive_pooling = AdaptiveConcatPool3d()
head = Head()

os.environ['CUDA_VISIBLE_DEVICES']='0,2,3'
device = torch.device('cuda') 

adaptive_pooling = adaptive_pooling.to(device)
head = head.to(device)
model.module.avgpool = adaptive_pooling
model.module.fc = head

checkpoint = torch.load('/media/hdd/astamoulakatos/save-model-3d/save_freezed_1.pth')
model.load_state_dict(checkpoint['state_dict'])
print('loading pretrained freezed model!')

for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for X, y in valid_loader:
        X = X.to(device)
        y = Variable(y.float()).to(device) 
        X = X.permute(0,2,1,3,4)
        y = y.squeeze(dim=1)
        y = y.float()
        output = model(X)
        y = y.detach().cpu()
        #loss = criterion(output, y)
        preds = torch.sigmoid(output)
        preds = preds.to(torch.float32) 
        preds = preds.detach().cpu()
        y_pred.append(preds)
        y_true.append(y)

In [ ]:
len(y_pred)

In [ ]:
y_tr = np.vstack([t.__array__() for tensor in y_true for t in tensor])

In [ ]:
len(y_tr)

In [ ]:
y_tr

In [ ]:
y_pr = np.vstack([t.__array__() for tensor in y_pred for t in tensor])

In [ ]:
import os
#import fastai
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, precision_recall_curve, accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import multilabel_confusion_matrix, precision_recall_fscore_support
import numpy as np

In [ ]:
y_pr

In [ ]:
# we assume that y contains a tuple of y_pred and targets
def nsea_compute_thresholds(y_true, y_pred):
#     y_pred = numpy.asarray(y[0])
#     y_true = numpy.asarray(y[1])
    precisions = dict()
    recalls = dict()
    Thresholds = dict()
    for i in range(5):
        precisions[i], recalls[i], Thresholds[i] = precision_recall_curve(y_true[:, i], y_pred[:, i])
    
    result = {}
    ###############
    ###############  FIX THE UGLY STAFF BELOW
    ###############
    classes = ['exp', 'bur', 'and', 'fj', 'fs']
    opt_id = []
    for i,event_type in enumerate(classes): 
        re = recalls[i]
        pre = precisions[i]
        dist = [ np.sqrt((1-re)**2 + (1-pre)**2) for re, pre in zip(re, pre) ]
        opt_id.append(dist.index(min(dist)))
        t = Thresholds[i]
        opt_thres = t[opt_id[i]]
        result[event_type] = opt_thres
    return result

In [ ]:
def new_compute_metrics(y_true, y_pred, thresholds):
    th = np.array([thresholds[key] for key in thresholds])
    classes = ['exp', 'bur', 'and', 'fj', 'fs']
    ## Apply digitisation on the outputs
    for idx, event_type in enumerate(classes):
        y_pred[:,idx] = np.where(y_pred[:,idx] >= thresholds[event_type], 1, 0)
    acc = []
    for idx, event in enumerate(classes):
        acc.append(accuracy_score(y_true[:,idx], y_pred[:, idx]))
    acc = np.array(acc)
    agg_acc = accuracy_score(y_true, y_pred)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred)
    
    cm=multilabel_confusion_matrix(y_true, y_pred)
    cmm=cm.reshape(-1,4)
    
    res_labels=pd.DataFrame({'Event': classes, 'Threshold': th, 'Exact Matching Score': acc, 'Precision': precision, 'Recall': recall, 'F1-Score': f1})
    
    res_labels = pd.concat([res_labels, pd.DataFrame(cmm, columns=['tn', 'fp', 'fn', 'tp'])], axis=1)
    
    agg_precision, agg_recall, agg_f1, agg_support = precision_recall_fscore_support(y_true, y_pred, average='micro')
    agg=pd.DataFrame({'Event': ['Aggregate'], 'Threshold': [np.nan], 'Exact Matching Score': agg_acc, 'Precision': agg_precision, 'Recall': agg_recall, 'F1-Score': agg_f1})
    
    
    agg = pd.concat([agg, pd.DataFrame(data=[np.nan]*4, index=['tn', 'fp', 'fn', 'tp']).T], axis=1)

    res=pd.concat([res_labels, agg]).reset_index(drop=True)
    
    return res

In [ ]:
thresholds = nsea_compute_thresholds(y_tr, y_pr)

In [ ]:
thresholds

In [ ]:
acc = accuracy_score(y_tr, y_pr)
f1 = f1_score(y_tr, y_pr, average="samples")


In [ ]:
acc, f1

In [ ]:
new_compute_metrics(y_tr, y_pr, thresholds)